In [15]:
import pandas as pd


file_path = "C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\NHTS Data Parser\\HH Data Parser\\trippub.csv"
df = pd.read_csv(file_path)

columns_to_keep = ['HOUSEID', 'PERSONID',"STRTTIME","ENDTIME",'TRVLCMIN', 'R_SEX', 'R_AGE', 'WRKCOUNT','HHVEHCNT', 'HHSIZE','EDUC','WHYTRP1S']
final_data = df[columns_to_keep].copy()




In [8]:
def translateAge(age):
    if age <=5:
        return 1
    elif age <= 18:
        return 2
    elif age <= 25:
        return 3
    elif age < 60:
        return 4
    else:
        return 5

# get the symbolic name of the gender from numerical value
def translateSex(sex):
    if sex == 1:
        return 'male'
    else:
        return 'female'

# get the race categories from numerical race values
def translateRace(race):
    if race == 1:
        return 1 #white
    elif race == 2:
        return 2 #black
    elif race == 4 and race ==6:
        return 3 #native
    elif race == 3:
        return 6 #asian
    elif race == 5:
        return 7 #'pacific-islander'
    else:
        return 8 #'other'
    
    

In [13]:

final_data['R_AGE'] = final_data.apply(lambda row: translateAge(row['R_AGE']), axis=1)
# final_data['R_RACE'] = final_data.apply(lambda row: translateRace(row['R_RACE']), axis=1)

In [14]:
final_data.head(10)

,HOUSEID,PERSONID,STRTTIME,ENDTIME,TRVLCMIN,R_SEX,R_AGE,WRKCOUNT,HHVEHCNT,HHSIZE,EDUC
0,30000007,1,1000,1015,15,2,5,1,5,3,3
1,30000007,1,1510,1530,20,2,5,1,5,3,3
2,30000007,2,700,900,120,1,5,1,5,3,3
3,30000007,2,1800,2030,150,1,5,1,5,3,3
4,30000007,3,845,900,15,2,4,1,5,3,2
5,30000007,3,1430,1445,15,2,4,1,5,3,2
6,30000008,1,1115,1130,15,1,4,2,4,2,5
7,30000008,1,2330,2340,10,1,4,2,4,2,5
8,30000012,1,550,605,15,2,4,1,2,1,5
9,30000012,1,700,715,15,2,4,1,2,1,5


In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
# X = final_data.drop(columns=['TRIPPURP','WHYFROM','HOUSEID', 'PERSONID',"TDAYDATE",'WRKCOUNT'],axis=1)
X = final_data
y = final_data['TRIPPURP']

# Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
label_encoder = LabelEncoder()
X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])


# Preprocess the target variable using LabelEncoder
y_label_encoder = LabelEncoder()
y_encoded = y_label_encoder.fit_transform(y)

# Convert the target variable to one-hot encoded format
num_classes = len(y_label_encoder.classes_)
y_one_hot = pd.get_dummies(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# X_train=X.astype('float64')
num_classes
# Standardize the features (optional, but often recommended for neural networks)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
input_shape = X_train.shape[1]

# X_test_scaled.shape
print(X_train_scaled.shape)
print(y_train.shape)
print(X.shape)
print(y.shape)

KeyError: "['WHYFROM', 'TDAYDATE'] not found in axis"

In [33]:
X_train_scaled

array([[0.36286562, 0.3819415 , 0.01157982, ..., 0.16666667, 0.25      ,
        0.76923077],
       [0.69944892, 0.70156846, 0.01157982, ..., 0.08333333, 0.08333333,
        0.92307692],
       [0.50869012, 0.51292921, 0.01571547, ..., 0.08333333, 0.41666667,
        0.53846154],
       ...,
       [0.29673591, 0.29885545, 0.01157982, ..., 0.16666667, 0.08333333,
        0.76923077],
       [0.86689275, 0.86901229, 0.01157982, ..., 0.16666667, 0.25      ,
        1.        ],
       [0.89020772, 0.89656634, 0.01985112, ..., 0.33333333, 0.16666667,
        0.92307692]])

In [19]:


# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# model = Sequential()

# # Add layers to the model
# model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=3, batch_size=32, validation_data=(X_test_scaled, y_test))
# model.fit(X_train, y, epochs=3, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/3
23090/23090 [==============================] - 17s 703us/step - loss: 1.4842 - accuracy: 0.3557 - val_loss: 1.4813 - val_accuracy: 0.3579
Epoch 2/3
23090/23090 [==============================] - 16s 711us/step - loss: 1.4808 - accuracy: 0.3567 - val_loss: 1.4792 - val_accuracy: 0.3581
Epoch 3/3
5773/5773 [==============================] - 3s 500us/step - loss: 1.4790 - accuracy: 0.3583
Test loss: 1.4789618253707886
Test accuracy: 0.35827627778053284


In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                576       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 6)                 198       
                                                                 
Total params: 2,854
Trainable params: 2,854
Non-trainable params: 0
_________________________________________________________________


In [125]:
X_train_scaled.shape

(923572, 8)

In [130]:
model.save("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\Model\\v1.h5")

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have X_train, X_test, y_train, y_test from the train-test split

# Create the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the model on the training data
dt_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = dt_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.25870665620009203
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.36      0.26      0.30     38092
           2       0.29      0.19      0.23     38881
           3       0.23      0.15      0.18     21993
           4       0.36      0.28      0.31     23445
           5       0.43      0.33      0.37     62299

   micro avg       0.36      0.26      0.30    184715
   macro avg       0.28      0.20      0.23    184715
weighted avg       0.35      0.26      0.30    184715
 samples avg       0.26      0.26      0.26    184715



C:\Users\abubabu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
# X = final_data.drop(columns=['TRIPPURP', 'WHYFROM'])
# y = final_data['WHYFROM']

# # Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
# label_encoder = LabelEncoder()
# X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
# X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# # Preprocess the target variable using LabelEncoder
# y_label_encoder = LabelEncoder()
# y_encoded = y_label_encoder.fit_transform(y)

# # Convert the target variable to one-hot encoded format
# num_classes = len(y_label_encoder.classes_)
# y_one_hot = pd.get_dummies(y_encoded)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# # Standardize the features (optional, but often recommended for neural networks)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)  # You can experiment with different learning rates
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Adding learning rate scheduler to reduce learning rate during training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model
model.fit(X_train_scaled, y_train, epochs=5, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/5
23090/23090 [==============================] - 24s 1ms/step - loss: 1.4564 - accuracy: 0.3737 - val_loss: 1.4104 - val_accuracy: 0.3911 - lr: 0.0010
Epoch 2/5
23090/23090 [==============================] - 23s 1ms/step - loss: 1.4317 - accuracy: 0.3852 - val_loss: 1.4005 - val_accuracy: 0.3997 - lr: 0.0010
Epoch 3/5
23090/23090 [==============================] - 26s 1ms/step - loss: 1.4267 - accuracy: 0.3878 - val_loss: 1.3996 - val_accuracy: 0.4021 - lr: 0.0010
Epoch 4/5
23090/23090 [==============================] - 26s 1ms/step - loss: 1.4239 - accuracy: 0.3885 - val_loss: 1.3982 - val_accuracy: 0.3972 - lr: 0.0010
Epoch 5/5
5773/5773 [==============================] - 3s 593us/step - loss: 1.3932 - accuracy: 0.4020
Test loss: 1.3931782245635986
Test accuracy: 0.4019652009010315


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

ValueError: y should be a 1d array, got an array of shape (738857, 6) instead.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
X = final_data.drop(columns=['TRIPPURP', 'WHYFROM'])
y = final_data['WHYFROM']

# Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
label_encoder = LabelEncoder()
X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# Preprocess the target variable using LabelEncoder
y_label_encoder = LabelEncoder()
y_encoded = y_label_encoder.fit_transform(y)

# Convert the target variable to one-hot encoded format
num_classes = len(y_label_encoder.classes_)
y_one_hot = pd.get_dummies(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Standardize the features (optional, but often recommended for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(BatchNormalization())  # Batch Normalization
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())  # Batch Normalization
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Define a learning rate scheduler
def lr_scheduler(epoch):
    return 0.001 * np.exp(-epoch / 10)

# Compile the model with SGD optimizer and learning rate scheduler
optimizer = SGD(learning_rate=0.0)  # Use a placeholder learning rate, the scheduler will update it
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Set up learning rate scheduler
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[lr_scheduler_callback])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/10
23090/23090 [==============================] - 33s 1ms/step - loss: 2.4161 - accuracy: 0.3158 - val_loss: 2.1299 - val_accuracy: 0.3470 - lr: 0.0010
Epoch 2/10
23090/23090 [==============================] - 32s 1ms/step - loss: 2.2169 - accuracy: 0.3409 - val_loss: 2.0955 - val_accuracy: 0.3499 - lr: 9.0484e-04
Epoch 3/10
23090/23090 [==============================] - 32s 1ms/step - loss: 2.1780 - accuracy: 0.3433 - val_loss: 2.0801 - val_accuracy: 0.3528 - lr: 8.1873e-04
Epoch 4/10
23090/23090 [==============================] - 32s 1ms/step - loss: 2.1586 - accuracy: 0.3441 - val_loss: 2.0729 - val_accuracy: 0.3537 - lr: 7.4082e-04
Epoch 5/10
23090/23090 [==============================] - 34s 1ms/step - loss: 2.1466 - accuracy: 0.3451 - val_loss: 2.0667 - val_accuracy: 0.3557 - lr: 6.7032e-04
Epoch 6/10
23090/23090 [==============================] - 32s 1ms/step - loss: 2.1394 - accuracy: 0.3455 - val_loss: 2.0637 - val_accuracy: 0.3560 - lr: 6.0653e-04
Epoch 7/10
23090/230

KeyboardInterrupt: 

In [ ]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler
from imblearn.over_sampling import RandomOverSampler

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
X = final_data.drop(columns=['TRIPPURP', 'WHYFROM'])
y = final_data['WHYFROM']

# Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
label_encoder = LabelEncoder()
X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# Preprocess the target variable using LabelEncoder
y_label_encoder = LabelEncoder()
y_encoded = y_label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features (optional, but often recommended for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Random Oversampling to balance class distribution
ros = RandomOverSampler(random_state=42)
X_train_scaled, y_train = ros.fit_resample(X_train_scaled, y_train)

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Define a learning rate scheduler
def lr_scheduler(epoch):
    return 0.001 * np.exp(-epoch / 10)

# Compile the model with RMSprop optimizer and learning rate scheduler
optimizer = RMSprop(learning_rate=0.0)  # Use a placeholder learning rate, the scheduler will update it
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Set up learning rate scheduler
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[lr_scheduler_callback])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

SyntaxError: invalid syntax (3506351552.py, line 66)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC



# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
X = final_data.drop(columns=['TRIPPURP','WHYFROM'])
y = final_data['WHYFROM']

final_data['WHYFROM'].head(10)



# Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
label_encoder = LabelEncoder()
X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# Preprocess the target variable using LabelEncoder
y_label_encoder = LabelEncoder()
y_encoded = y_label_encoder.fit_transform(y)

# Convert the target variable to one-hot encoded format
num_classes = len(y_label_encoder.classes_)
y_one_hot = pd.get_dummies(y_encoded)

# Split the data into training and testing sets (use the same split as the neural network)
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Standardize the features (optional, but often recommended for some classifiers)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)






# Decision Tree
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train_scaled, y_train)
decision_tree_accuracy = decision_tree_model.score(X_test_scaled, y_test)
print("Decision Tree accuracy:", decision_tree_accuracy)

# Random Forest
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train_scaled, y_train)
random_forest_accuracy = random_forest_model.score(X_test_scaled, y_test)
print("Random Forest accuracy:", random_forest_accuracy)

# Support Vector Machine (SVM)
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)
svm_accuracy = svm_model.score(X_test_scaled, y_test)
print("SVM accuracy:", svm_accuracy)


NameError: name 'final_data' is not defined

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
X = final_data.drop(columns=['TRIPPURP'])
y = final_data['TRIPPURP']

# Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
label_encoder = LabelEncoder()
X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# Preprocess the target variable using LabelEncoder
y_label_encoder = LabelEncoder()
y_encoded = y_label_encoder.fit_transform(y)

# Convert the target variable to one-hot encoded format
num_classes = len(y_label_encoder.classes_)
y_one_hot = pd.get_dummies(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a function to build the neural network model
def create_model(optimizer='adam', activation='relu', dropout_rate=0.2, hidden_units=64):
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=X_train_scaled.shape[1], activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(hidden_units, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Wrap the Keras model in a scikit-learn estimator
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define hyperparameters to search over
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'activation': ['relu', 'leakyrelu'],
    'dropout_rate': [0.2, 0.3],
    'hidden_units': [64, 128]
}

# Perform GridSearchCV for hyperparameter tuning
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best hyperparameters and model
best_params = grid_result.best_params_
best_model = grid_result.best_estimator_

# Train the best model on the full training set
best_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test_scaled, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

C:\Users\Rahat Khan\AppData\Local\Temp\ipykernel_11408\3243449808.py:53: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


KeyboardInterrupt: 